In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import warnings

C:\Users\david\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Filter out the specific warning about feature names not matching
warnings.filterwarnings("ignore", message="X does not have valid feature names, but NearestNeighbors was fitted with feature names")

In [3]:
# Load the movie-item matrix from the CSV file
user_item_matrix = pd.read_csv('./data_for_ml.csv')

# 1. Handling Missing Values - dropping rows with missing values
user_item_matrix.dropna(inplace=True)

# 2. Normalizing Numerical Features
scaler = MinMaxScaler()
numerical_cols = ['Year', 'average_rating', 'rating_count']
user_item_matrix[numerical_cols] = scaler.fit_transform(user_item_matrix[numerical_cols])

# 3. Encoding Categorical Variables (Genre columns in this case)
genre_cols = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 
              'Drama', 'Fantasy', 'Horror', 'IMAX', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'Western']
encoder = OneHotEncoder(sparse_output=False)
encoded_genres = encoder.fit_transform(user_item_matrix[genre_cols])

# Create feature names for the encoded genres
feature_names = []
for category_list in encoder.categories_:
    feature_names.extend(category_list)

# Create a DataFrame for the encoded genres with proper column names
encoded_genres_df = pd.DataFrame(encoded_genres, columns=feature_names)
user_item_matrix_encoded = pd.concat([user_item_matrix.drop(columns=genre_cols), encoded_genres_df], axis=1)

In [4]:
# Initialize the K-Nearest Neighbors Model 
knn_movie_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='cosine')

# Define the Recommendation Algorithm Function
def recommendation_algorithm(movie_name, matrix, movie_cf_model, n_recs):
    # Fit model on matrix
    movie_cf_model.fit(matrix.drop(['title', 'Year', 'average_rating', 'rating_count'], axis=1, errors='ignore'))
    
    # Try to find a close match for the movie name input using FuzzyWuzzy search
    match = process.extractOne(movie_name, matrix.index)
    print("Match:", match)  # Error handling statement if movie is not found
    if match and match[1] > 70:  # Lowered similarity threshold
        movie_name_matched = match[0]
    else:
        return "Movie not found or not a close match."
   
    # Get the movie title for the matched index
    matched_movie_title = matrix.loc[movie_name_matched, 'title']
    print("Matched Movie Title:", matched_movie_title)  # Print the matched movie title
    
    # Get the movie row based on the matched movie name
    movie_row = matrix.loc[movie_name_matched]
    
    # Calculate neighbor distances
    distances, indices = movie_cf_model.kneighbors(movie_row.drop(['title', 'Year', 'average_rating', 'rating_count'], errors='ignore').values.reshape(1, -1), n_neighbors=n_recs)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    # List to store recommendations
    movie_recs = []
    for i in movie_rec_ids:
        movie_recs.append({'Movie ID': matrix.iloc[i[0]].name, 'Distance': i[1]})
    
    # Display the movie titles for the recommended movies
    recommended_movies = pd.DataFrame(movie_recs)
    recommended_movies['Title'] = recommended_movies['Movie ID'].apply(lambda x: matrix.loc[x, 'title'])
    
    # Select top number of recommendations needed
    recommended_movies = recommended_movies[['Title', 'Distance']].head(n_recs)
    
    return recommended_movies


In [5]:
# Print the matched movie title and a list of 10 recommended movies with their distance to the input, sorted by Distance
recommended_movies = recommendation_algorithm("#Alive (2020)", user_item_matrix, knn_movie_model, 11)
recommended_movies.sort_values("Distance").reset_index(drop=True)

Match: (20, 90)
Matched Movie Title: Gravity (2013)


,Title,Distance
0,The Amazing Spider-Man 2 (2014),0.000000
1,Gravity (2013),0.000000
2,Men in Black III (M.III.B.) (M.I.B.³) (2012),0.133975
3,Tron: Legacy (2010),0.133975
4,Transformers: Dark of the Moon (2011),0.133975
5,Resident Evil: Retribution (2012),0.133975
6,Transformers: Revenge of the Fallen (2009),0.133975
7,Captain America: The Winter Soldier (2014),0.133975
8,"Amazing Spider-Man, The (2012)",0.133975
9,Star Wars: Episode II - Attack of the Clones (...,0.133975
